In [7]:
from typing import List
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.outputs import ChatResult, ChatGeneration
import google.generativeai as genai
import os

# --- Configure Google API ---
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

from pydantic import BaseModel, Field

class ClarifyWithUser(BaseModel):
    """Schema for user clarification decisions during scoping phase."""
    need_clarification: bool = Field(
        description="Whether the user needs to be asked a clarifying question.",
    )
    question: str = Field(
        description="A question to ask the user to clarify the report scope",
    )
    verification: str = Field(
        description="Verify message that we will start research after the user has provided the necessary information.",
    )

# --- Minimal Gemma Wrapper ---
class ChatGemma(BaseChatModel):
    model: str = "gemma-3-4b-it"

    def _generate(
        self, messages: List[BaseMessage], **kwargs
    ) -> ChatResult:
        # Convert LangChain messages to plain text
        prompt = "\n".join(
            f"{m.type.upper()}: {m.content}" for m in messages
        )
        resp = genai.GenerativeModel(self.model).generate_content(prompt)

        return ChatResult(
            generations=[ChatGeneration(message=AIMessage(content=resp.text))]
        )

    @property
    def _llm_type(self) -> str:
        return "gemma"

# --- Example usage ---
model = ChatGemma()

# 1. With structured outputs
structured_model = model.with_structured_output(ClarifyWithUser)

# 2. Bind tools
from langchain_core.tools import tool

@tool
def add(x: int, y: int) -> int:
    """Add two numbers"""
    return x + y

model_with_tools = structured_model.bind_tools([add])

# --- Run ---
response = model_with_tools.invoke(
    [HumanMessage(content="Use the add tool to add 7 and 5.")]
)

print(response)


NotImplementedError: with_structured_output is not implemented for this model.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai

# Configure with your API key
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

# Initialize Gemma model
model = genai.GenerativeModel("gemma-3-4b-it")

# Run inference
prompt = "Explain large language models in simple terms."
response = model.generate_content(prompt)

print(response.text)


In [ ]:
from rich.markdown import Markdown
Markdown(response.text)

In [5]:
from pydantic import BaseModel, Field

class ClarifyWithUser(BaseModel):
    """Schema for user clarification decisions during scoping phase."""
    need_clarification: bool = Field(
        description="Whether the user needs to be asked a clarifying question.",
    )
    question: str = Field(
        description="A question to ask the user to clarify the report scope",
    )
    verification: str = Field(
        description="Verify message that we will start research after the user has provided the necessary information.",
    )

model = model.with_structured_outputs(ClarifyWithUser)

AttributeError: 'GenerativeModel' object has no attribute 'with_structured_outputs'

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI
from pydantic import BaseModel, Field

from typing import List
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.outputs import ChatResult, ChatGeneration
import google.generativeai as genai
import os

class ClarifyWithUser(BaseModel):
    need_clarification: bool = Field(description="Whether clarification is needed")
    question: str = Field(description="Clarifying question for the user")
    verification: str = Field(description="Verification message after clarification")

# --- Minimal Gemma Wrapper ---
class ChatGemma(BaseChatModel):
    model: str = "gemma-3-27b-it"
    # Configure with your API key
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    # Initialize Gemma model
    model = genai.GenerativeModel("gemma-3-27b-it")

    def _generate(
        self, messages: List[BaseMessage], **kwargs
    ) -> ChatResult:
        # Convert LangChain messages to plain text
        prompt = "\n".join(
            f"{m.type.upper()}: {m.content}" for m in messages
        )
        resp = genai.GenerativeModel(self.model).generate_content(prompt)

        return ChatResult(
            generations=[ChatGeneration(message=AIMessage(content=resp.text))]
        )

    @property
    def _llm_type(self) -> str:
        return "gemma"
# --- Example usage ---
model = ChatGemma()

# 1. With structured outputs
structured_model = model.with_structured_output(ClarifyWithUser)

# 2. Bind tools
from langchain_core.tools import tool

@tool
def add(x: int, y: int) -> int:
    """Add two numbers"""
    return x + y

model_with_tools = structured_model.bind_tools([add])

# --- Run ---
response = model_with_tools.invoke(
    [HumanMessage(content="Use the add tool to add 7 and 5.")]
)

print(response)

response = structured_model.invoke("Do you need more info before writing the report?")
print(response)


NotImplementedError: with_structured_output is not implemented for this model.

In [1]:
from langchain_core.tools import tool
from pydantic import BaseModel

@tool
def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y


In [ ]:
class SumResult(BaseModel):
    sum: int



In [4]:
from dotenv import load_dotenv
load_dotenv()
import os

from langgraph.prebuilt import create_react_agent
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize Gemma-3-27B-IT locally (replace with your HF token if needed)
gemma_llm = ChatGoogleGenerativeAI(
    model="gemma-3-27b-it",
    temperature=0,
    api_key=os.getenv("GEMINI_API_KEY")
)

# Create the agent with tool and structured output schema
agent = create_react_agent(
    model=gemma_llm,
    tools=[add],
    prompt="You are a helpful assistant.",
    response_format=SumResult
)


In [5]:
# Example run
input_state = {"messages": [{"role": "user", "content": "Add 15 and 27"}]}
result = agent.invoke(input_state)

print(result["structured_response"])  # -> SumResult(sum=42)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Function calling is not enabled for models/gemma-3-27b-it